In [1]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI
from langchain.chains import LLMChain
import textwrap
from langchain.chains import LLMSummarizationCheckerChain
import os
from PyPDF2 import PdfReader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import FAISS
from langchain.chains import RetrievalQA

os.environ["OPENAI_API_KEY"] = #INSERT YOUR KEY HERE#

In [11]:
def split_text(raw_text):
    text_splitter = CharacterTextSplitter(        
        separator = "\n",
        chunk_size = 2500,
        chunk_overlap  = 400, #striding over the text
        length_function = len,
    )
    texts = text_splitter.split_text(raw_text)
    return texts

raw_text = """"The document discusses the annual government accounts of the Municipality of Amarante for the year 2017. The report highlights various observations and issues, including discrepancies in the presentation of debt and floating debt, positive and negative aspects of the Transparency Portal, and the need for improvement in certain sectors such as education and planning. The Ministry of Public Accounts recommends the approval of the accounts with reservations.
- The report discusses the accounts of the Municipal Government of Amarante for the year 2017.
- The Demonstrative of Internal Funded Debt and the Floating Debt Statement do not provide complete information on previous balances and future balances.
- The Transparency Portal of the government has both positive and negative aspects.
- The Ministry of Public Accounts recommends approving the accounts with reservations.
- The municipality of Amarante has achieved below-average scores in the i-Educ index but above-average scores in the i-Saúde, i-Gov TI, and i-Fiscal indices.
- The municipality has not met the projected goals for education in the years 2011-2017.
- The report highlights several issues, including late submission of budgetary documents, absence of COSIP in the Annual Budget Law, and negative values in the "Maximum of 5% not applied in the exercise" indicator.
"""


texts = split_text(raw_text)

prompt_merge = """"Role: You are an auditor from civil society interest in public accounts.
You know this context from these two sources in plain text and in list:{context}
Here is your task: {question}
Use only information from the sources. Do not add external information or include opnions.
Include all information available in the answer.
Here is the answer:"""

prompt_merge_check = PromptTemplate(template=prompt_merge, input_variables=['context', 'question'])

embeddings = OpenAIEmbeddings()
docsearch = FAISS.from_texts(texts, embeddings)      
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})
rqa = RetrievalQA.from_chain_type(llm=OpenAI(), 
                                  chain_type="stuff", 
                                  retriever=retriever,
                                  chain_type_kwargs={"prompt": prompt_merge_check},
                                  return_source_documents=True)
question = "Merge the results providing a plain text summary with deeper details using only the information from these two sources"
question = "Include all information of the list on the plain text to expand and merge the information of both sources."

In [12]:
resposta_LLM = rqa(question)

In [13]:
resposta_LLM['result']

' The document discusses the annual government accounts of the Municipality of Amarante for the year 2017. The report highlights various observations and issues, including discrepancies in the presentation of debt and floating debt, with the Demonstrative of Internal Funded Debt and the Floating Debt Statement not providing complete information on previous balances and future balances, and the need for improvement in certain sectors such as education and planning. The Transparency Portal of the government has both positive and negative aspects, with the municipality of Amarante having achieved below-average scores in the i-Educ index but above-average scores in the i-Saúde, i-Gov TI, and i-Fiscal indices. The municipality has not met the projected goals for education in the years 2011-2017, and the report highlights several issues, including late submission of budgetary documents, absence of COSIP in the Annual Budget Law, and negative values in the "Maximum of 5% not applied in the ex

In [10]:
len(' The annual government accounts of the Municipality of Amarante for the year 2017 have been discussed in the report. It highlights discrepancies in the presentation of debt and floating debt and has identified both positive and negative aspects of the Transparency Portal. The Ministry of Public Accounts has recommended approval of the accounts with reservations. The municipality has achieved below-average scores in the i-Educ index but above-average scores in the i-Saúde, i-Gov TI, and i-Fiscal indices. The report also states that the municipality has not met projected goals for education in the years 2011-2017, due to late submission of budgetary documents, absence of COSIP in the Annual Budget Law, and negative values in the "Maximum of 5% not applied in the exercise" indicator.')

791